Use this utlity to update the returns and std_dev fields within investment-options.csv

In [1]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import brownbear as bb

# Format price data.
pd.options.display.float_format = '{:0.2f}'.format

%matplotlib inline

In [3]:
# Set size of inline plots.
'''note: rcParams can't be in same cell as import matplotlib
   or %matplotlib inline
   
   %matplotlib notebook: will lead to interactive plots embedded within
   the notebook, you can zoom and resize the figure
   
   %matplotlib inline: only draw static images in the notebook
'''
plt.rcParams["figure.figsize"] = (10, 7)

Globals

In [4]:
# Set refresh_timeseries=True to download timeseries.  Otherwise /symbol-cache is used.
refresh_timeseries = True

In [5]:
# Read in sp400.csv
sp400 = pd.read_csv('sp400.csv')
sp400.drop(columns=['SEC filings'], inplace=True)
sp400.rename(columns={'Ticker symbol': 'Symbol',
                      'Security':'Description',
                      'GICS Sector':'Asset Class',
                      'GICS Sub-Industry': 'GICS Sub Industry'}, inplace=True)
sp400.set_index("Symbol", inplace=True)
sp400

,Description,Asset Class,GICS Sub Industry
Symbol,,,
AA,Alcoa,Materials,Aluminum
ACC,American Campus Communities,Real Estate,Residential REITs
ACHC,Acadia Healthcare,Health Care,Health Care Facilities
ACIW,ACI Worldwide,Information Technology,Application Software
ACM,AECOM,Industrials,Construction & Engineering
...,...,...,...
XRX,Xerox,Information Technology,"Technology Hardware, Storage & Peripherals"
Y,Alleghany Corporation,Financials,Reinsurance
YELP,Yelp,Communication Services,Interactive Media & Services


In [6]:
# Read in gics-2-asset-class.csv
gics2asset_class = pd.read_csv('gics-2-asset-class.csv', skip_blank_lines=True, comment='#')
gics2asset_class.set_index("GICS", inplace=True)
gics2asset_class = gics2asset_class['Asset Class'].to_dict()
gics2asset_class

{'Energy': 'US Stocks:Energy',
 'Materials': 'US Stocks:Materials',
 'Industrials': 'US Stocks:Industrials',
 'Consumer Discretionary': 'US Stocks:Consumer Discretionary',
 'Consumer Staples': 'US Stocks:Consumer Staples',
 'Health Care': 'US Stocks:Healthcare',
 'Financials': 'US Stocks:Financials',
 'Information Technology': 'US Stocks:Technology',
 'Communication Services': 'US Stocks:Communication Services',
 'Utilities': 'US Stocks:Utilities',
 'Real Estate': 'US Stocks:Real Estate'}

In [7]:
# Map sp400 GICS sectors to brownbear defined asset classes.
def _asset_class(row):
    return gics2asset_class[row['Asset Class']]

sp400['Asset Class'] = sp400.apply(_asset_class, axis=1)

# Yahoo finance uses '-' where '.' is used in symbol names.
sp400.index = sp400.index.str.replace('.', '-', regex=False)
sp400

,Description,Asset Class,GICS Sub Industry
Symbol,,,
AA,Alcoa,US Stocks:Materials,Aluminum
ACC,American Campus Communities,US Stocks:Real Estate,Residential REITs
ACHC,Acadia Healthcare,US Stocks:Healthcare,Health Care Facilities
ACIW,ACI Worldwide,US Stocks:Technology,Application Software
ACM,AECOM,US Stocks:Industrials,Construction & Engineering
...,...,...,...
XRX,Xerox,US Stocks:Technology,"Technology Hardware, Storage & Peripherals"
Y,Alleghany Corporation,US Stocks:Financials,Reinsurance
YELP,Yelp,US Stocks:Communication Services,Interactive Media & Services


In [8]:
# Drop invalid symbols.
#sp400.drop(['AAXN'], inplace=True)

In [9]:
# Make symbols list.
symbols = list(sp400.index)
#symbols

In [10]:
# Get the timeseries for the symbols and compile into a single csv.
bb.fetch_timeseries(symbols, refresh=refresh_timeseries)
bb.compile_timeseries(symbols)

................................................................................................................................................................................................................................................................................................................................................................................................................


In [11]:
# Read symbols timeseries into a dataframe.
df = pd.read_csv('symbols-timeseries.csv', skip_blank_lines=True, comment='#')
df.set_index("Date", inplace=True)
df = df[:]
df

,AA,ACC,ACHC,ACIW,ACM,ADNT,AEO,AFG,AGCO,AIRC,...,WU,WWD,WWE,X,XPO,XRX,Y,YELP,YETI,ZD
Date,,,,,,,,,,,,,,,,,,,,,
2015-01-02,37.16,31.25,59.98,19.71,30.29,NaN,11.06,38.03,41.14,NaN,...,13.84,46.35,10.78,24.98,23.65,27.21,444.64,55.15,NaN,53.26
2015-01-05,35.01,31.44,59.12,19.26,28.96,NaN,11.19,37.53,39.25,NaN,...,13.71,45.35,10.58,23.82,22.92,26.62,435.03,52.53,NaN,52.40
2015-01-06,35.27,31.76,58.19,18.95,28.67,NaN,11.23,37.31,39.06,NaN,...,13.69,45.02,10.61,23.09,22.11,26.26,432.47,52.44,NaN,51.97
2015-01-07,36.18,31.81,60.63,19.03,29.26,NaN,11.78,37.56,39.04,NaN,...,13.56,45.44,10.17,23.15,22.20,26.60,434.10,52.21,NaN,52.17
2015-01-08,37.21,32.00,61.76,19.01,30.17,NaN,11.22,38.24,39.87,NaN,...,13.83,45.21,10.05,23.66,22.32,27.33,437.79,53.83,NaN,53.20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-04-18,89.45,57.58,73.80,27.82,77.15,34.03,17.66,146.75,141.74,51.76,...,18.94,117.43,59.86,37.16,55.02,19.04,839.31,34.60,54.40,96.39
2022-04-19,86.58,64.80,74.35,28.56,78.34,35.91,17.78,146.02,143.29,52.89,...,19.24,120.70,60.27,37.78,56.35,19.56,837.28,35.09,56.32,99.11
2022-04-20,86.93,64.75,75.91,29.03,78.05,36.74,17.69,148.98,144.76,54.84,...,19.40,123.20,60.78,36.91,57.19,19.85,837.01,34.53,55.82,98.01


In [12]:
# Sample symbol.
symbol = 'ACHC'

In [13]:
annual_returns = bb.annualized_returns(df, timeperiod='daily', years=1)
annual_returns[symbol]

9.358423601929378

In [14]:
# Calculate 1 month, 3 months, 1 year, 3 year, and 5 year annualized returns.
annual_returns_1mo = bb.annualized_returns(df, timeperiod='daily', years=1/12)
annual_returns_3mo = bb.annualized_returns(df, timeperiod='daily', years=3/12)
annual_returns_1yr = bb.annualized_returns(df, timeperiod='daily', years=1)
annual_returns_3yr = bb.annualized_returns(df, timeperiod='daily', years=3)
annual_returns_5yr = bb.annualized_returns(df, timeperiod='daily', years=5)

In [15]:
# Calculate 20 day annualized volatility.
daily_returns = df.pct_change()
years = bb.TRADING_DAYS_PER_MONTH / bb.TRADING_DAYS_PER_YEAR
vola = bb.annualized_standard_deviation(daily_returns, timeperiod='daily', years=years)
vola[symbol]

0.44767083817632586

In [16]:
# Calculate 20 day annualized downside volatility.
ds_vola = bb.annualized_standard_deviation(daily_returns, timeperiod='daily', years=years, downside=True)
ds_vola[symbol]

0.33080425999784085

In [17]:
# Resample df on a monthly basis.
df.index = pd.to_datetime(df.index)
monthly = df.resample('M').ffill()

In [18]:
# Calculate monthly returns.
monthly_returns = monthly.pct_change()
monthly_returns[symbol]

Date
2015-01-31     NaN
2015-02-28    0.09
2015-03-31    0.13
2015-04-30   -0.04
2015-05-31    0.08
              ... 
2021-12-31    0.08
2022-01-31   -0.13
2022-02-28    0.08
2022-03-31    0.16
2022-04-30    0.04
Freq: M, Name: ACHC, Length: 88, dtype: float64

In [19]:
# Calculate 3 year annualized standard deviation.
std_dev = bb.annualized_standard_deviation(monthly_returns, timeperiod='monthly', years=3)
std_dev[symbol]

0.44363813392080464

In [20]:
# Read investment-options-header.csv
lines = []
with open('investment-options-in.csv', 'r') as f:
    lines = [line.strip() for line in f]
#lines

In [21]:
# For each symbol, write out the 1 Yr, 3 Yr, 5 Yr, and std dev.
out = lines.copy()

# This is still slow (2.53 s).
for i, (index, row) in enumerate(sp400.iterrows()):

    symbol = index
    description = row['Description']
    asset_class = row['Asset Class']

    ret_1mo = annual_returns_1mo[symbol]
    ret_3mo = annual_returns_3mo[symbol]
    ret_1yr = annual_returns_1yr[symbol]
    ret_3yr = annual_returns_3yr[symbol]
    ret_5yr = annual_returns_5yr[symbol]
    
    if np.isnan(ret_3yr): ret_3yr = ret_1yr
    if np.isnan(ret_5yr): ret_5yr = ret_3yr

    _vola = vola[symbol]*100
    _ds_vola = ds_vola[symbol]*100
    sd = std_dev[symbol]*100

    out.append(
        '"{}","{}","{}","{:0.2f}","{:0.2f}","{:0.2f}","{:0.2f}","{:0.2f}","{:0.2f}","{:0.2f}","{:0.2f}"'
        .format(symbol, description, asset_class,
                ret_1mo, ret_3mo, ret_1yr, ret_3yr, ret_5yr, _vola, _ds_vola, sd)) 

In [22]:
# Write out asset-classes.csv
with open('investment-options.csv', 'w') as f:
    for line in out:
        f.write(line + '\n')